In [1]:
import torch 
from torch.utils.data import DataLoader,Dataset
import pandas as pd
import numpy as np

In [2]:
class dataset(Dataset):
    def __init__(self,data,Type,N_size):
        super().__init__()
        index = np.lexsort((data[:,2],))
        data = data[index]
        data = np.array(data)
        if N_size == '<126':
            data = data[np.where(data[:,2]<126)]
        else:
            data = data[np.where(data[:,2]>=126)]
        
        if Type == 'odd':
            data = data[np.intersect1d(np.where(data[:,2]%2==1),np.where(data[:,1]%2==1) )]
        elif Type == 'even':
            data = data[np.intersect1d(np.where(data[:,2]%2==0),np.where(data[:,1]%2==0))]
        else:
            data = data[np.where((data[:,2]+data[:,1])%2==1)]
        
        
        self.data = data[:,:5].astype(np.float32)
        self.target = data[:,-4].astype(np.float32)
        
        
    def __getitem__(self,idx):
        return self.data[idx],self.target[idx]
    
    def __len__(self):
        return len(self.data)
    

In [3]:
def get_Kfold(file,k,shuffle=True):
    data = pd.read_excel(file,engine="openpyxl")
    data = np.array(data)
    if shuffle:
        np.random.shuffle(data) 
    num = len(data)//k
    return [(np.concatenate((data[0:num*i],data[num*(i+1):len(data)])),data[num*i:num*(i+1)]) for i in range(k)]

In [4]:
file = './data/totaldata.xlsx'
Type = 'odd'
N_size = '<126'

K_fold_data = get_Kfold(file,5)
for trian_data,test_data in K_fold_data:
    trian_data = dataset(trian_data,Type,N_size)
    test_data = dataset(test_data,Type,N_size)
    train_loader = DataLoader(dataset=trian_data,batch_size=32)
    test_loader = DataLoader(dataset=test_data,batch_size=32)
    